In [0]:
from google.colab import drive

In [0]:
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#content/gdrive/My Drive/AIML/INTERNAL RESIDENCY6


In [0]:
import tensorflow as tf

In [0]:
#Reset Default graph - Needed only for Jupyter notebook
tf.reset_default_graph()

**collect** data

In [0]:
import pandas as pd


In [0]:
#!ls "/content/gdrive/My Drive/AIML/INTERNAL RESIDENCY6"
#data=pd.read_csv("content/gdrive/My Drive/AIML/INTERNAL RESIDENCY6/Iris.csv")
from google.colab import files
uploaded = files.upload()

Saving Iris.csv to Iris.csv
Saving prices.csv to prices.csv


In [0]:
import io
df = pd.read_csv(io.StringIO(uploaded['prices.csv'].decode('utf-8')))
df.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


Check all columns in the dataset

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


Drop columns date and symbol

In [0]:
df.drop(df.columns[[0, 1]], axis=1, inplace=True)

In [0]:
df.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


Consider only first 1000 rows in the dataset for building feature set and target set

In [0]:
df2 = df.head(1000)

Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
import numpy as np
y= np.arange(1000)

X_train, X_test, y_train, y_test = train_test_split(df2, y, test_size=0.2)

Building the graph in tensorflow

In [0]:
#Input features
x = tf.placeholder(shape=[None,5],dtype=tf.float32, name='x-input')

#Normalize the data
x_n = tf.layers.batch_normalization(x,training=True)

#Actual Prices
y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y-input')

In [0]:
#Define Weights and Bias
W = tf.Variable(tf.zeros(shape=[5,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

In [0]:
#prediction
y = tf.add(tf.matmul(x_n,W),b,name='output')

In [0]:
#Loss (Cost) Function [Mean square error]
loss = tf.reduce_mean(tf.square(y-y_),name='Loss')

In [0]:
#GradientDescent Optimizer to minimize Loss
train_op = tf.train.GradientDescentOptimizer(0.003).minimize(loss)

Executing the Graph


In [0]:
#Lets start graph Execution
sess = tf.Session()

# variables need to be initialized before we can use them
sess.run(tf.global_variables_initializer())

#how many times data need to be shown to model
training_epochs = 100

In [0]:
#Load data

#(features, actual_prices),_ = df2.load_data(test_split=0)

features=df2
actual_prices=np.arange(1000)

In [0]:
for epoch in range(1000):
            
    #Calculate train_op and loss
    _, train_loss = sess.run([train_op,loss],feed_dict={x:features, 
                                                        y_:actual_prices})
    
    if epoch % 10 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss)

Training loss at step:  0  is  332833.53
Training loss at step:  10  is  304535.16
Training loss at step:  20  is  nan
Training loss at step:  30  is  nan
Training loss at step:  40  is  nan
Training loss at step:  50  is  nan
Training loss at step:  60  is  nan
Training loss at step:  70  is  nan
Training loss at step:  80  is  nan
Training loss at step:  90  is  nan
Training loss at step:  100  is  nan
Training loss at step:  110  is  nan
Training loss at step:  120  is  nan
Training loss at step:  130  is  nan
Training loss at step:  140  is  nan
Training loss at step:  150  is  nan
Training loss at step:  160  is  nan
Training loss at step:  170  is  nan
Training loss at step:  180  is  nan
Training loss at step:  190  is  nan
Training loss at step:  200  is  nan
Training loss at step:  210  is  nan
Training loss at step:  220  is  nan
Training loss at step:  230  is  nan
Training loss at step:  240  is  nan
Training loss at step:  250  is  nan
Training loss at step:  260  is  nan


In [93]:
sess.run(y, feed_dict={x:features[0:1]})

array([[nan]], dtype=float32)

In [97]:
sess.run(y, feed_dict={x:[[1,2,3,400,5],
                          [1,2,3,400,5]]})

array([[nan],
       [nan]], dtype=float32)

Linear Classification using Keras

Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
Use Mean square error as loss function and sgd as optimize

In [98]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from keras.utils import to_categorical

Using TensorFlow backend.


In [100]:
dataset = pd.read_csv(io.StringIO(uploaded['Iris.csv'].decode('utf-8')))
dataset.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [0]:
#Splitting the data into training and test test
X = dataset.iloc[:,1:5].values
y = dataset.iloc[:,5].values

from sklearn.preprocessing import LabelEncoder
encoder =  LabelEncoder()
y1 = encoder.fit_transform(y)

Y = pd.get_dummies(y1).values


from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

In [102]:
#Defining the model 

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD,Adam


model = Sequential()

model.add(Dense(10,input_shape=(4,),activation='tanh'))
model.add(Dense(8,activation='tanh'))
model.add(Dense(6,activation='tanh'))
model.add(Dense(3,activation='softmax'))

model.compile(Adam(lr=0.04),'categorical_crossentropy',metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                50        
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 88        
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 54        
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 21        
Total params: 213
Trainable params: 213
Non-trainable params: 0
_________________________________________________________________


In [103]:
#fitting the model and predicting 
model.fit(X_train,y_train,epochs=100)
y_pred = model.predict(X_test)

y_test_class = np.argmax(y_test,axis=1)
y_pred_class = np.argmax(y_pred,axis=1)

Epoch 1/100
120/120 [==============================] - 0s 3ms/step - loss: 1.1426 - acc: 0.3917
Epoch 2/100
120/120 [==============================] - 0s 206us/step - loss: 1.0968 - acc: 0.4333
Epoch 3/100
120/120 [==============================] - 0s 230us/step - loss: 0.9846 - acc: 0.5333
Epoch 4/100
120/120 [==============================] - 0s 225us/step - loss: 0.8049 - acc: 0.6917
Epoch 5/100
120/120 [==============================] - 0s 212us/step - loss: 0.6129 - acc: 0.7000
Epoch 6/100
120/120 [==============================] - 0s 213us/step - loss: 0.5004 - acc: 0.7000
Epoch 7/100
120/120 [==============================] - 0s 204us/step - loss: 0.4471 - acc: 0.9417
Epoch 8/100
120/120 [==============================] - 0s 225us/step - loss: 0.4178 - acc: 0.8000
Epoch 9/100
120/120 [==============================] - 0s 228us/step - loss: 0.3895 - acc: 0.8000
Epoch 10/100
120/120 [==============================] - 0s 240us/step - loss: 0.2962 - acc: 0.8917
Epoch 11/100
120/120 

In [104]:
#Accuracy of the predicted values
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test_class,y_pred_class))
print(confusion_matrix(y_test_class,y_pred_class))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       0.93      1.00      0.96        13
           2       1.00      0.83      0.91         6

   micro avg       0.97      0.97      0.97        30
   macro avg       0.98      0.94      0.96        30
weighted avg       0.97      0.97      0.97        30

[[11  0  0]
 [ 0 13  0]
 [ 0  1  5]]
